# Задание 4

## 1. Chainer (0.4 балла)

Требуется реализовать функцию chainer, которая принимает на вход любое количество итерируемых объектов (в т.ч. контейнеров, которые тоже могут содержать итерируемые объекты) и возвращает список из всех элементов всех переданных контейнеров.

In [1]:
def check_if_iterable(item):
    try:
        it = iter(item)
        return len(item) > 1 or not isinstance(item, str)
    except TypeError as te:
        return False

def chainer(*args):
    answer = []
    
    def flatten(iterable):
        
        def get_item(item):
            
            if check_if_iterable(item):
                flatten(item)
            else:
                answer.append(item)
        
        list(map(get_item, iterable))
    
    flatten(args)
    return answer

Пример работы функции:

In [2]:
chainer([1, 2, 3], "why", ["i need", ["do that", ["?"]]])

[1,
 2,
 3,
 'w',
 'h',
 'y',
 'i',
 ' ',
 'n',
 'e',
 'e',
 'd',
 'd',
 'o',
 ' ',
 't',
 'h',
 'a',
 't',
 '?']

При этом запрещается использовать в коде конструкции циклов while и for.

## 2. Dataflow (0.6 балла)

В данном задании требуется написать обработчик генераторов, вычисляющих факториал. Для каждого генератора должна быть функция декоратор, которая инициализирует генератор.

In [3]:
from functools import wraps, reduce

In [4]:
def coroutine(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        generator = func(*args, **kwargs)
        generator.send(None)
        return generator
    
    return wrapper

Функция генератор факториалов принимает на вход n - максимальный факториал, который может вычислить (n!).

In [5]:
@coroutine
def fact_generator(n):
    res = 1
    multiplier = 0
    
    while multiplier <= n:
        value = yield res
        if value is None:
            multiplier += 1
            res *= multiplier
        else:
            if value > multiplier:
                res *= reduce(lambda x, y: x * y, range(multiplier + 1, value + 1))
                multiplier = value
            else:
                raise StopIteration

Пример:

f10 = fact_generator(10)

создает генератор, который при каждом вызове next(f10) возвращает текущее значение факториала, т. е.

next(f10) # вернет 1

next(f10) # вернет 2

next(f10) # вернет 6

...


При этом должен быть реализован функционал, который позволяет послать значение z в генератор и должно вернуться соответствующее значение факториала последовательным применением функции next. Если это невозможно (генератор исчерпался), должно быть выведено сообщение об этом и последнее возможное значение факториала.

Данный функционал должен быть реализован в классе GeneratorManeger:

In [6]:
class GeneratorManager:
    
    class Generator:
        
        def __init__(self, generator, max_value=1):
            self.generator = generator(max_value)
            self.max_value = max_value
            self.counter = 0
        
        def __next__(self):
            self.counter += 1
            return next(self.generator)
        
        def __call__(self):
            return self
        
        def __iter__(self):
            return self
        
        def send(self, value):
            return self.generator.send(value)
            
            
    def __init__(self, *fact_desc):
        self.generators = [GeneratorManager.Generator(fact_generator, value) for value in fact_desc]
    
    def send(self, z):
        matching_generator = None
        
        for gen in self.generators:
            if gen.counter < z <= gen.max_value:
                matching_generator = gen
                break
        if matching_generator is None:
            print("There isn't a suitable generator!")
            return None
        matching_generator.counter = z
        return matching_generator.send(z)

fact_desc - это список из чисел $n_i$ для каждого из которых создается свой генератор fact_generator($n_i$) и сохраняется в классе GeneratorManeger.

send должна выбрать любой генератор из еще не исчерпавшихся и вычислить значение факториала. Если таких нет, сообщить об этом.

Пример работы:

In [7]:
gm = GeneratorManager(3, 6)

In [8]:
gm.send(3)

6

In [9]:
gm.send(3)

6

In [10]:
gm.send(3)

There isn't a suitable generator!


In [11]:
gm.send(5)

120

In [12]:
gm.send(6)

720

In [13]:
gm.send(1)

There isn't a suitable generator!
